In [1]:
import numpy as np
import math

### Activation Functions

$$linear(z) = z$$
$$relu(z) = max(0, z)$$
$$sigmoid(z) = 1 / (1 + e^{-z})$$
$$tanh(z) = 2/(1+e^{-2z})-1$$
$$softplus(z) = ln(1+e^{z})$$


In [ ]:
def act_function(z,activation):
    """
    Inputs:
        z : z can either be a number or a numpy array
        activation: Python string denoting the activation function to be used in the current layer 
    Outputs:
        Activation function applied to z; if z is an array, the function must be applied to its each element
    """
    if activation == "linear":
        return z
    elif activation == "relu":
        return z*(z>0)
    elif activation == "sigmoid":
        return 1 / (1 + np.exp(-1*z))
    elif activation == "tanh":
        return (2 / (1 + np.exp(-2*z))) -1
    elif activation == "softplus":
        return np.log(1+ np.exp(z))
        

### Initializing the Weights
The NN will contain $H-1$ hidden layers and an output layer. For a layer $h$, we will have two parameters/variables:
  * Weight matrix $W^{[h]}$(of shape (size of current layer, size of previous layer)),
  * Bias vector $b^{[h]}$(of shape (size of current layer, 1)),  

*   Used np.random.uniform(low, high, shape), with low=0.0 and high=0.01 to initialize the weights $W^{[h]}$ (<a href="https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.uniform.html#numpy-random-uniform">np.random.uniform()</a>).

*   Initialized biases $b^{[h]}$ with zeros (<a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html">np.zeros()</a>).

* For a single-hidden layer NN with input of size 5, hidden layer size = 4 and output layer size =3, the input parameter ```layers_sizes``` will be **[5, 4, 3]**. The output of the ```construct_NN()``` function will be  **```[[np.array(shape=(4, 5)), np.array(shape=(4, 1))], [np.array(shape=(3, 4)), np.array(shape=(3, 1))]]```**, with each numpy array initialised as mentioned above.

In [4]:
def construct_NN(layers_sizes):
    """
    Inputs:
        layers_sizes: A list (of length H+1) containing the size (number of neurons) of each layer. Element at index 0 represents input layer size.  
    Outputs:
        weights: A list (of length H), where each element is a list - [Wh, bh], representing the  weights matrix and bias vector of layer h (Both Wh and bh are numpy arrays).
    """
    low = 0
    high = 0.01
    H = len(layers_sizes)-1
    weights = []
    for h in range(H):
        w = np.random.uniform(low, high, (layers_sizes[h+1],layers_sizes[h]))
        b = np.zeros((layers_sizes[h+1],1))
        weights.append([w,b])
    return weights 

### Forward Propagation
This function carries out computations of a single layer of the neural network:
 $$ Z^{[h]} = W^{[h]}.A^{[h-1]}+b^{[h]}$$
 $$ A^{[h]} = g^{[h]}(Z^{[h]}) $$

In [ ]:
def forward_step(A_prev, W, b, activation):
    """
    Inputs:
        A_prev: Input data OR Previous layer's activations of shape (size of previous layer, number of examples)
        W: Weight matrix of shape (size of current layer, size of previous layer)
        b: Bias vector of shape (size of the current layer, 1)
        activation: Python string denoting the activation function to be used in the current layer 
    Outputs:
        A: Current layer's activations of shape (size of current layer, number of examples)
        Z: Current layer's pre activation value, i.e. input of activation function
    """
    Z = W.dot(A_prev)+b
    A = act_function(Z, activation)  
    return A, Z

###  Forward Propagation Module
* This function iterates over the $H$ layers of the neural network using a for loop, computing the final output of the neural network. 
* It also outputs a ```layers_cache``` list (of length H), whose each element corresponds to a layer $h$ and is a list[$A^{[h-1]}, W^{[h]}, Z^{[h]}$]. Storing these values now will be helpful while computing gradients n the backward pass.
* For the sake of clarity, consider a single-hidden layer NN with input of size 5, hidden layer size = 4 and output layer size =3. For this example, the output ```layers_cache``` list will be [[$X or A^{[0]}$, $W^{[1]}$, $Z^{[1]}$], [$A^{[1]}$, $W^{[2]}$, $Z^{[2]}$]].

In [ ]:
def forward_module(X, weights, activations):
    """
    Inputs:
        X: Input data of shape (size of input layer, number of examples)
        weights: A list (of length H), where each element is a list - [Wh, bh], representing the weights matrix and bias vector of layer h (Both Wh and bh are numpy arrays).
        activations: A list (of length H), where each element is a Python string representing the activation function to be used in the corresponding layer
    Outputs:
        AH: Output layer's activations
        layers_cache: A list (of length H), where each element is a list [A_prev, W, Z], representing the values A_prev (input to the layer), W (Weight matrix of the layer)
          and Z (pre-activation value) for layer h. The purpose of cache is to store values that will be needed during backpropagation.
    """
    layers_cache = []
    H = len(weights)
    A = X

    for h in range(H):
        activation = activations[h]
        #     print('{}, weight {}'.format(h, weights[h][0].shape))
        _,Z = forward_step(A, weights[h][0],weights[h][1], activation, alpha)
        layers_cache.append([A,weights[h][0],Z])
        A,_ = forward_step(A, weights[h][0],weights[h][1], activation, alpha)

    return A, layers_cache

###  Cost Function
Compute the cross-entropy cost using the following formula: $$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}*\log\left(a^{[H] (i)}\right) + (1-y^{(i)})*\log\left(1- a^{[H](i)}\right))$$

Compute the mean square error cost using the following formula: $$\frac{1}{m} \sum\limits_{i = 1}^{m} \left(y^{(i)} - \left(a^{[H] (i)}\right)\right)^2 $$
where  
$$m = number \ of \ examples$$
$$ y^{(i)} = Ground \ truth \ labels \ for \ i-th \ example $$
$$ a^{[H](i)} = Output \ layer \ activations \ for \ i-th \ example $$
$$ * \ denotes \ elementwise \ multiplication $$


In [ ]:
def cost_function(AH, Y,error_function,error_function):
    """
    Inputs:
        AH: Activations of output layer, representing probability vector corresponding to your label predictions, of shape (1, number of examples)
        Y: Ground truth "label" vector of shape (1, number of examples)
        error_function: Python string denoting the error function to be used
    Outputs:
        cost: Cost of shape () i.e. scalar value
    """
    m = Y.shape[1]
    if error_function == "cross_entropy":
        cost = (-1/m)*((np.multiply(Y,np.log(AH)) + np.multiply(1 - Y, np.log(1-AH))).sum())
    elif error_function == "mean_square"
      cost = (1/m)*np.sum(np.square(Y-AH))
    
    return cost

###  Activation Backward Propagation
This function computes  $dZ^{[h]} = \frac{\partial \mathcal{L} }{\partial Z^{[h]}}$, given  $dA^{[h]} = \frac{\partial \mathcal{L} }{\partial A^{[h]}}$ and $Z^{[h]}$as inputs. It is based on the following chain rule from calculus:
$$ dZ^{[h]} = dA^{[h]} * \frac{\partial \mathcal{A^{[h]}} }{\partial Z^{[h]}}$$ <br>

Note the following rules for different activation functions:
$$ \frac{\partial \mathcal{(relu(z))} }{\partial z} =   \left\{
\begin{array}{ll}
      1 & z>=0 \\
      0& z < 0 \\
\end{array} 
\right.  $$<br>
$$\frac{\partial \mathcal{(sigmoid(z))} }{\partial z} = sigmoid(z) * (1-sigmoid(z))$$<br>
$$\frac{\partial \mathcal{(tanh(z))} }{\partial z} = 1-tanh(z)*tanh(z)$$<br>
$$\frac{\partial \mathcal{(softplus(z))} }{\partial z} = sigmoid(z) $$

In [ ]:
def act_backward(dA, Z,activation):
    """
    Inputs:
        dA: Gradient of the cost function w.r.t. the post activation value A, of shape(size of current layer, number of examples)
        Z: Pre activation value, of shape(size of current layer, number of examples)
        activation: Python string denoting the activation function to be used in the current layer 
    Outputs:
        dZ: Gradient of the cost function w.r.t. Z, of shape(size of current layer, number of examples)
    """
    if activation == "linear":
        dZ = dA
    elif activation == "relu":
        dZ = dA * (Z>=0)
    elif activation == "sigmoid":
        dZ = dA * sigmoid(Z)*(1- sigmoid(Z))
    elif activation == "tanh":
        dZ = np.multiply(dA, (1- tanh(Z)*tanh(Z)))
    elif activation == "softplus":
        dZ = dA * sigmoid(Z)
    return dZ


### Backward Propagation Step
* The following function computes $dA^{[h-1]}, dW^{[h]}$ and $db^{[h]}$, given $dA^{[h]}, Z^{[h]}, W^{[h]}$ and $A^{[h-1]}$ as inputs.  
* We can calculate $dZ^{[h]}$ using $dA^{[h]}$ and $Z^{[h]}$ as inputs to the ```activation_backward()``` function defined above.  
* The three outputs $(dA^{[h-1]}, dW^{[h]}, db^{[h]})$ are computed using $dZ^{[h]}, W^{[h]}$ and $A^{[h-1]}$ using the below equations(using the fact that $ Z^{[h]} = W^{[h]}.A^{[h-1]}+b^{[h]}$):  
  - $ dA^{[h-1]} = \frac{\partial \mathcal{L} }{\partial A^{[h-1]}} = W^{[h] T} dZ^{[h]}$
  - $ dW^{[h]} = \frac{\partial \mathcal{L} }{\partial W^{[h]}} = \frac{1}{m} dZ^{[h]} A^{[h-1] T}$
  - $ db^{[h]} = \frac{\partial \mathcal{L} }{\partial b^{[h]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[h](i)}$


In [19]:
def backward_step(dA, layer_cache, activation):
    """
    Inputs:
        dA: Gradient of the cost function w.r.t. the post activation value A for current layer
        layer_cache: A list [A_prev, W, Z] for current layer, where A_prev is activation of prev layer, W is weight matrix of current layer, and Z is pre-activation value for current layer
        activation: Python string denoting the activation function used in the current layer
    Outputs:
        dA_prev: Gradient of the cost function w.r.t. the activation (of the previous layer h-1), of same shape as A_prev
        dW: Gradient of the cost function w.r.t. W (current layer l), of same shape as W
        db: Gradient of the cost function w.r.t. b (current layer l), of same shape as b
    """
    A_prev = layer_cache[0]
    W = layer_cache[1]
    Z = layer_cache[2]

    m = Z.shape[1]
    dZ = activation_backward(dA,Z,activation,alpha)
    dA_prev = np.matmul((W.T),dZ)
    dW = np.matmul(dZ,A_prev.T)/m
    db = np.sum(dZ,axis =1,keepdims = True)/m
    return dA_prev, dW, db

### Backward Propagation Module 
* This function computes the gradients of loss function w.r.t. all the weights of the neural network.
* To backpropagate through this network, we know that the output is, $A^{[H]} = sigmoid(Z^{[H]})$. Your code thus needs to compute $ dA^{[h]} = \frac{\partial \mathcal{L}}{\partial A^{[H]}}$. To do so, use this formula:
$$ dA^{[h]} = -(Y / A^{[h]} - (1-Y)/(1-A^{[h]}))$$
where $ "/" \ denotes \ elementwise \ division$
* The output of this function is a list ```gradients``` of length $H$, whose each element is a list [$dW^{[h]}, db^{[h]}$].
* For the sake of clarity, consider a single-hidden layer NN with input of size 5, hidden layer size = 4 and output layer size =3. For this example, the output ```gradients``` will be [[$dW^{[1]}, db^{[1]}$], [$dW^{[2]}, db^{[2]}$]].

In [20]:
### 1.5 Marks
def backward_module(AH, Y, layers_cache, activations):
    """
    Inputs:
        AH: Activations of output layer, representing probability vector corresponding to your label predictions, of shape (1, number of examples)
        Y: Ground truth "label" vector of shape (1, number of examples)
        layers_cache - A list (of length H), where each element is a list [A_prev, W, Z], representing the values A_prev (input to the layer), W (Weight matrix of the layer)
          and Z (pre-activation value) for layer h. The purpose of cache is to store values that will be needed during backpropagation.
        activations - A list (of length H), where each element is a Python string representing the activation function to be used in the corresponding layer

    Outputs:
        gradients: A list (of length H), where each element is a list [dW, db], representing the values dW (gradient of cost function w.r.t. W) and  
          db (gradient of cost function w.r.t. b)
    """
    gradients = []
    H = len(layers_cache) # the number of layers
    m = AH.shape[1]
    Y = Y.reshape(AH.shape) 
    dAH = -(Y/AH - (1-Y)/(1-AH))
    dA = dAH
    for i in reversed(range(H)):
        dA_prev, dW, db = backward_step(dA,layers_cache[i],activations[i],alpha)
        gradients.insert(0,(dW,db))
        dA = dA_prev

    return gradients

### Batch Gradient Descent Optimizer
Now that we have calculated the gradients of the loss function w.r.t. all the weights of the network, update the weights of the model, using gradient descent: 

$$ W^{[h]} = W^{[h]} - \alpha \text{ } dW^{[h]} $$
$$ b^{[h]} = b^{[h]} - \alpha \text{ } db^{[h]} $$

where $\alpha$ is the learning rate.   
After computing the updated weights, store them back in the weights list. 

In [21]:
def update_weights(weights, gradients, lr):
    """
    Inputs:
        weights: A list (of length H), where each element is a list - [Wh, bh], representing the weights matrix and bias vector of layer h (Both Wh and bh are numpy arrays).
        gradients: A list (of length H), where each element is a list - [dWh, dbh], representing the gradients of cost function w.r.t. weights matrix and bias vector 
          of layer h respectively (Both dWh and dbh are numpy arrays).
        lr: Learning Rate.
    Outputs:
        weights: A list (of length H), containing the updated weights.
    """
    H = len(weights)
    for h in range(H):
        weights[h][0] -= lr*gradients[h][0]
        weights[h][1] -= lr*gradients[h][1]
        
    return weights

### Train a H-layer Neural Network:
This function performs the following steps:  
1. Initialize the weights of the neural network .
2. Perform forward propagation on input argument ```X```.
3. Compute cross entropy loss function.
4. Perform backward propagation to compute gradients of loss function w.r.t. all the weights of the network.
5. Use the gradients computed in step 4 to update the weights of the network.
6. Append cost's value to the list ```epoch_wise_costs```.
7. Print (epoch, cross entropy cost's value) for epoch = 0, 100, 200, ...
8. Repeat steps 2-7 for ```epochs```(input argument to below function) number of iterations.
9. Return the weights of the neural network.


In [ ]:
def H_layer_NN(X, Y, layers_sizes, activations, epochs=200, lr=0.001, error_function="mean_square"):
    """
    Inputs:
        X: Input data, numpy array of shape (number_of_features_in_input_data, number of examples)
        Y: Ground truth "label" vector of shape (1, number of examples)
        layers_sizes: A list (of length H+1) containing the size (number of neurons) of each layer. Element at index 0 represents input layer size.
        activations: A list (of length H), where each element is a Python string representing the activation function to be used in the corresponding layer
        epochs: number of epochs of the training loop
        lr: learning rate of the gradient descent update rule
        error_function: Python string denoting the error function to be used
    Outputs:
        weights - weights learnt by the model (can be used for prediction). A list (of length H), where each element is a list - [Wh, bh], representing the weights matrix and bias vector of layer h.
    """
    epoch_wise_costs = []
    
    weights = construct_NN(layers_sizes)
    for epoch in range(epochs):
        AH, layers_cache = forward_module(X, weights, activations, alpha)
        cost = cross_entropy_cost(AH, Y,error_function)
        gradients = backward_module(AH, Y, layers_cache, activations, alpha)
        update_weights(weights, gradients, lr)
        epoch_wise_costs.append(cost)
        if epoch % 100 == 0:
            print("Cost after epoch %i: %f" %(epoch, cost))
    
    plt.plot(np.array(epoch_wise_costs))
    plt.ylabel('Cost Function')
    plt.xlabel('Number of Epochs')
    plt.title("Training Curve")
    
    return weights

# Currently Binary Prediction. Need to change!

In [ ]:
def predict(X, Y, weights, activations, alpha=0.1, mode="Test",NN_type):
    """
    Inputs:
        X: Input data/examples you would like to predict
        Y: Ground truth labels corresponding to input data X
        weights: weights of the trained model
        activations: A list (of length H), where each element is a Python string representing the activation function to be used in the corresponding layer
        mode: Test/Training mode
        NN_type: Normal/ Wavelet Architecture
    Outputs:
        p: predictions for the given dataset X
    """
    m = X.shape[1]
    p = np.zeros(m)
    AH, _ = forward_module(X, weights, activations, alpha)
    AH = np.squeeze(AH)
    pos_pred = AH > 0.5
    p[pos_pred] = 1
    print(NN_type +" Neural Network: " + mode + " Accuracy: "  + str(np.sum(p == Y)/m))
    return p

### Main Function

In [ ]:
layers_sizes = [12, 25, 1]
epochs = 10000
lr = 0.03
error_function = "mean_square"
activations_normal = ["leaky_relu", "sigmoid"]
activations_wavelet = [] ###DO
weights_normal = H_layer_NN(X_train, Y_train, layers_sizes, activations_normal, epochs, lr, error_function)
weights_wavelet = H_layer_NN(X_train, Y_train, layers_sizes, activations_wavelet, epochs, lr, error_function)

### Results

In [ ]:
training_predictions_normal = predict(X_train, Y_train, weights_normal, activations_normal, mode="Train","Normal")
test_predictions_normal = predict(X_test, Y_test, weights_normal, activations_normal, mode="Test", "Normal")

training_predictions_wavelet = predict(X_train, Y_train, weights_wavelet, activations_wavelet, mode="Train","Wavelet")
test_predictions_wavelet = predict(X_test, Y_test, weights_wavelet, activations_wavelet, mode="Test", "Wavelet")

# TO DO: 
* Wavelet Functions
* Fix Prediction Functions
* Set Model Architecture
* Train and Compare